In [8]:
import pandas as pd
import yaml
import torch
import os
from pytorch_lightning import Trainer, seed_everything
from eyemind.trainer.loops import KFoldLoop
import eyemind
from eyemind.models.transformers import InformerEncoderDecoderModel, InformerEncoderFixationModel, InformerMultiTaskEncoderDecoder
from eyemind.dataloading.informer_data import InformerDataModule


In [9]:
import matplotlib.patches as mpatches



def fixation_image(pred,target, title="Fixation Identification"):
    sl = len(pred)
    fixation_labels = torch.cat((pred.expand(sl//2,sl),target.expand(sl//2,sl)))
    plt.imshow(fixation_labels, extent=[0, len(fixation_labels[1]),0, 100], cmap='Greys')
    # plt.xticks(np.arange(0, len(fixation_labels), 1), [])
    plt.yticks([])
    # plt.grid(True, axis='x', lw=1, c='black')
    # plt.tick_params(axis='x', length=0)
    plt.title(title)
    black = mpatches.Patch(color='black', label='Fixation')
    white = mpatches.Patch(color='white', label='Saccade')
    plt.legend(handles=[black, white],bbox_to_anchor=(1.15, 1), loc='upper right')
    plt.xlabel("Time Steps (100 steps ~ 1.7s)")
    plt.show()

In [24]:
repodir = os.path.dirname(os.path.dirname(eyemind.__file__))

for fold in [0, 1, 2, 3]:
    save_dir = f"{repodir}/lightning_logs/informer_pretraining_seed21/fold{fold}/"
    config_path=os.path.join(save_dir,"config.yaml")
    ckpt_path = os.path.join(save_dir,"checkpoints","last.ckpt")
    with open(config_path, "r") as f:
        config = yaml.safe_load(f)

    model = InformerMultiTaskEncoderDecoder.load_from_checkpoint(ckpt_path,
                                                    encoder_weights_path=None)
    # model.tasks=["fi"]
    model.eval()

    # print(model.tasks)
    # trainer = Trainer(**config["trainer"])
    seed_everything(config["seed_everything"], workers=True)
    data_dir = os.path.join(repodir,config["data"]["data_dir"])
    label_file = os.path.join(repodir,config["data"]["label_filepath"])
    config["data"]["data_dir"]=data_dir
    config["data"]["label_filepath"]=label_file
    
    datamodule = InformerDataModule(**config["data"])
    datamodule.setup()

    test_dl = datamodule.get_dataloader(datamodule.test_dataset)
    for i,batch in enumerate(test_dl):
        print(f"batch: {i}")
        print(f"length of batch: {len(batch)}")
        preds = model(batch)
        print(f"preds: {preds.shape}")
        if i==1:
            break
        fixation_preds = preds["fi"]
        fixation_targets = batch[1]
        fixation_image(fixation_preds[1], fixation_targets[1], "Fixation Identification - Informer")
    # pc_targets = batch[0][:4,-100:]
    # rc_targets = batch[0][:4,:-100]

/usr/local/Caskroom/miniconda/base/envs/dg/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AveragePrecision` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
Global seed set to 21


batch: 0
length of batch: 4


TypeError: InformerMultiTaskEncoderDecoder.forward() missing 1 required positional argument: 'x_dec'

In [21]:
with torch.no_grad():

    logits = model.forward(batch[0])
    print(logits.shape)
#     601 def _step(self, batch, batch_idx, step_type):
#     602     try:
# --> 603         X, fix_y, X2, cl_y = batch
#     604     except ValueError as e:
#     605         print(f"{batch}")
# ValueError: not enough values to unpack (expected 4, got 2)

# preds = model(batch[0], batch[1], i) 
# # 'InformerMultiTaskEncoderDecoder' object has no attribute 'decoder'

# preds = model.predict_step(batch[0],batch[1], i) 
# # TypeError: InformerMultiTaskEncoderDecoder.forward() missing 1 required positional argument: 'x_dec'

# preds = model.predict(batch, i) 
# # AttributeError: 'InformerMultiTaskEncoderDecoder' object has no attribute 'predict'


TypeError: InformerMultiTaskEncoderDecoder.forward() missing 1 required positional argument: 'x_dec'

In [17]:
preds = model.forward(batch[0],batch[1])
print(f"preds: {preds.shape}")
fixation_preds = preds["fi"]
fixation_targets = batch[1]
fixation_image(fixation_preds[1], fixation_targets[1], "Fixation Identification - Informer")
# pc_targets = batch[0][:4,-100:]
# rc_targets = batch[0][:4,:-100]



AttributeError: 'InformerMultiTaskEncoderDecoder' object has no attribute 'decoder'

TODO:
try loading encoder and fi decoder separately
try modifying train loop to store the preds per batch
within loop, get batch and run thru 2 models - ensure ssame random subsequence chosen

In [ ]:
# COmprehension: Rote X
